In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg12-rddBasics").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg12-rddBasics").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_16447


In [ ]:
spark.range(10).rdd


# COMMAND ----------

spark.range(10).toDF("id").rdd.map(lambda row: row[0])


# COMMAND ----------

spark.range(10).rdd.toDF()


# COMMAND ----------

myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)


# COMMAND ----------

words.setName("myWords")
words.name() # myWords


# COMMAND ----------

def startsWithS(individual):
  return individual.startswith("S")


# COMMAND ----------

words.filter(lambda word: startsWithS(word)).collect()


# COMMAND ----------

words2 = words.map(lambda word: (word, word[0], word.startswith("S")))


# COMMAND ----------

words2.filter(lambda record: record[2]).take(5)


# COMMAND ----------

words.flatMap(lambda word: list(word)).take(5)


# COMMAND ----------

words.sortBy(lambda word: len(word) * -1).take(2)


# COMMAND ----------

fiftyFiftySplit = words.randomSplit([0.5, 0.5])


# COMMAND ----------

spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x, y: x + y) # 210


# COMMAND ----------

def wordLengthReducer(leftWord, rightWord):
  if len(leftWord) > len(rightWord):
    return leftWord
  else:
    return rightWord

words.reduce(wordLengthReducer)


# COMMAND ----------

words.getStorageLevel()


# COMMAND ----------

words.mapPartitions(lambda part: [1]).sum() # 2


# COMMAND ----------

def indexedFunc(partitionIndex, withinPartIterator):
  return ["partition: {} => {}".format(partitionIndex,
    x) for x in withinPartIterator]
words.mapPartitionsWithIndex(indexedFunc).collect()


# COMMAND ----------

spark.sparkContext.parallelize(["Hello", "World"], 2).glom().collect()
# [['Hello'], ['World']]


# COMMAND ----------

